# Title of Your Group Project

This Python notebook serves as a template for your group project for the course "Modeling in Cognitive Science".

This is the practical part of the group project where you get to implement the computational modeling workflow. In this part, you are expected to:


*   Implement at least two computational models relevant for your hypothesis. *(3 points)*
*   Simulate behavior from the two models. *(3 points)*
*   Implement a procedure for fitting the models to data. *(4 points)*
*   Implement a procedure for parameter recovery. *(5 points)*
*   (Implement a procedure for model recovery.) *(optional; 2 bonus points)*
*   Implement a model comparison. *(5 points)*.

You can gain a total of 20 points for the practical part of the group project.

**Note:** *Some of the exercises below (e.g. Model Simulation) rely on code from previous exercises (e.g., Model Implementation). In such cases, you are encouraged to rely on functions implemented for previous exercises. That is, you don't have to produce redundant code.*



## Model Implementation *(3 points)*

For this exercise you should:

*   Implement and simulate data from two* models that are suitable to test your hypothesis. *(3 points)*

<font size=2>*You may implement more than two models if you wish. However, two models are sufficient for this group project.</font>

Make sure to comment your code and provide an explanation for each code block in a preceding text block.


In [8]:
from simulate import simulate
# Run simulation and save to csv
agent_type = 'model_based'
simulate(agent_type=agent_type)

qtable:
 [[0.42946679 0.32925158]
 [0.51340353 0.16164472]
 [0.22395175 0.33645644]]
transition model for relevant states-action:
 [[0.         0.71698113 0.28301887]
 [0.         0.30851064 0.69148936]]
transition model for other states-actions:
 [[[0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]]]


,common_transition,state_transition_to,reward_stage_1,action_stage_1,reward_stage_2,action_stage_2,reward_probabilities,trial_index
0,False,1,0,1,1,1,"[0.0, 0.0, 0.33046909620385206, 0.738363678605...",0
1,False,2,0,0,1,0,"[0.0, 0.0, 0.32555543097151307, 0.744012723148...",1
2,False,1,0,1,1,1,"[0.0, 0.0, 0.29590509048928715, 0.730574468994...",2
3,True,2,0,1,1,0,"[0.0, 0.0, 0.2624516658310027, 0.7139690522468...",3
4,True,2,0,1,1,0,"[0.0, 0.0, 0.27702484170566366, 0.699147946027...",4
...,...,...,...,...,...,...,...,...
195,True,2,0,1,0,1,"[0.0, 0.0, 0.45475382535434977, 0.497073205862...",195
196,False,1,0,1,0,0,"[0.0, 0.0, 0.4651556773651332, 0.4954945293717...",196
197,True,1,0,0,1,0,"[0.0, 0.0, 0.47732583780026233, 0.487225764491...",197
198,True,1,0,0,1,0,"[0.0, 0.0, 0.45889353858713605, 0.510680073443...",198


In [9]:
import pandas as pd
import numpy as np
import os


# Load latest simulated data from csv
data_folder = os.path.join("data", "simulated", agent_type)
timestamped_folders = os.listdir(data_folder)
timestamped_folders.sort()
latest_folder = timestamped_folders[-1]
filename = os.path.join(data_folder, latest_folder, "simulated_data.csv")
print("Loading data from", filename)
task_df = pd.read_csv(filename)

# print some statistics 
# TODO stay probabilities?
print("common transitions percentage:", np.mean(task_df["common_transition"])*100, "%")
print("rewarded trails percentage:", np.mean(task_df["reward_stage_2"] > 0)*100, "%")
print("transition prob. from state 0 action 0 to state 1:", np.mean(task_df[task_df["action_stage_1"] == 0]["state_transition_to"] == 1)*100, "%")
print("transition prob. from state 0 action 1 to state 2:", np.mean(task_df[task_df["action_stage_1"] == 1]["state_transition_to"] == 2)*100, "%")

visited_states_counts_df = task_df['state_transition_to'].value_counts().reset_index()
visited_states_counts_df.columns = ['State', 'Counts']
display(visited_states_counts_df)

action_first_stage_counts_df = task_df['action_stage_1'].value_counts().reset_index()
action_first_stage_counts_df.columns = ['Action Stage 1', 'Counts']
display(action_first_stage_counts_df)


display(task_df)


Loading data from data/simulated/model_based/20240218-151136/simulated_data.csv
common transitions percentage: 70.5 %
rewarded trails percentage: 42.0 %
transition prob. from state 0 action 0 to state 1: 71.69811320754717 %
transition prob. from state 0 action 1 to state 2: 69.14893617021278 %


,State,Counts
0,1,105
1,2,95


,Action Stage 1,Counts
0,0,106
1,1,94


,common_transition,state_transition_to,reward_stage_1,action_stage_1,reward_stage_2,action_stage_2,reward_probabilities,trial_index
0,False,1,0,1,1,1,[0. 0. 0.3304691 0.73836368 0...,0
1,False,2,0,0,1,0,[0. 0. 0.32555543 0.74401272 0...,1
2,False,1,0,1,1,1,[0. 0. 0.29590509 0.73057447 0...,2
3,True,2,0,1,1,0,[0. 0. 0.26245167 0.71396905 0...,3
4,True,2,0,1,1,0,[0. 0. 0.27702484 0.69914795 0...,4
...,...,...,...,...,...,...,...,...
195,True,2,0,1,0,1,[0. 0. 0.45475383 0.49707321 0...,195
196,False,1,0,1,0,0,[0. 0. 0.46515568 0.49549453 0...,196
197,True,1,0,0,1,0,[0. 0. 0.47732584 0.48722576 0...,197
198,True,1,0,0,1,0,[0. 0. 0.45889354 0.51068007 0...,198


In [10]:
# load and inspect human data
human_data = pd.read_csv("data/two_steps_experiment_data_muhip.csv")
display(human_data)

# print some statistics
common_transitions_percentage = np.mean(
    np.where(human_data["stepOneChoice"] == 0, human_data["isHighProbOne"], human_data["isHighProbTwo"])
) * 100
print("common transitions percentage:", common_transitions_percentage, "%")
print("rewarded trails percentage:", np.mean(human_data["reward"] > 0)*100, "%")

,stepOne_Param,stepOneTwo_Param,stepTwoTwo_Param,rewards_Param,stepOneChoice,stepTwoChoice,reward,rewardProbabilities,isHighProbOne,isHighProbTwo,trial_type,trial_index,time_elapsed,internal_node_id
0,"[1,0]","[0,1]","[3,2]","[false,true,true,true]",1,3,True,"[0.6091146172868657,0.4713462443008218,0.61151...",True,True,dawsTwoStep,0,11309,0.0-0.0
1,"[1,0]","[0,1]","[2,3]","[true,true,true,false]",1,3,False,"[0.6059896178088179,0.49839269509234124,0.6221...",True,True,dawsTwoStep,1,16301,0.0-1.0
2,"[1,0]","[1,0]","[2,3]","[true,true,true,true]",1,2,True,"[0.5656024832486587,0.45986309794430447,0.6272...",True,True,dawsTwoStep,2,20980,0.0-2.0
3,"[1,0]","[1,0]","[1,0]","[false,true,false,false]",0,0,False,"[0.5605234469272922,0.4597340637004513,0.63268...",True,False,dawsTwoStep,3,25341,0.0-3.0
4,"[0,1]","[1,0]","[3,2]","[true,false,true,false]",1,2,True,"[0.6010152805775881,0.46606593103274335,0.6537...",True,True,dawsTwoStep,4,29309,0.0-4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,"[1,0]","[2,3]","[1,0]","[true,false,true,true]",0,3,True,"[0.611406687652392,0.36566104490101065,0.66043...",False,False,dawsTwoStep,195,719537,0.0-195.0
196,"[0,1]","[2,3]","[3,2]","[false,false,true,false]",0,3,False,"[0.6011729983654986,0.33707366326489835,0.7135...",False,True,dawsTwoStep,196,722105,0.0-196.0
197,"[1,0]","[1,0]","[1,0]","[true,true,true,true]",1,1,True,"[0.5389335487015099,0.34496395128204227,0.7221...",True,False,dawsTwoStep,197,724543,0.0-197.0
198,"[1,0]","[0,1]","[2,3]","[false,false,true,false]",0,1,False,"[0.5570029346925695,0.3068807208779234,0.74131...",True,True,dawsTwoStep,198,726632,0.0-198.0


common transitions percentage: 67.0 %
rewarded trails percentage: 59.5 %


In [11]:
from environment import TwoStepEnv
# simple one run test
# np.random.seed(0)
env = TwoStepEnv()

terminal = False
while not terminal:
    action = np.random.choice([0,1])
    s, r, terminal, info = env.step(action)

print(env.info)

{'common_transition': True, 'state_transition_to': 1, 'reward_stage_1': 0, 'action_stage_1': 0, 'reward_stage_2': 0, 'action_stage_2': 1, 'reward_probabilities': array([0.        , 0.        , 0.36185563, 0.34923468, 0.31881933,
       0.40339384])}


## Model Simulation *(3 points)*

For this exercise you should:

*   Simulate data from both models for a single set of parameters. The simulation should mimic the experiment you are trying to model. *(2 points)*

*   Plot the simulated behavior of both models. *(1 point)*

Make sure to comment your code and provide an explanation for each code block in a preceding text block.


In [ ]:
# YOUR MODEL SIMULATION CODE GOES HERE

## Parameter Fitting *(4 points)*

For this exercise you should:

*   Set up a suitable parameter search space *(1 point)*

*   Implement a procedure to evaluate the fit of a model based on data *(2 points)*

*   Implement a procedure for searching the parameter space. *(1 point)*

Make sure to comment your code and provide an explanation for each code block in a preceding text block.



In [ ]:
# YOUR PARAMETER FITTING CODE GOES HERE

## Parameter Recovery *(5 points)*

For this exercise you should:

*   Set up a suitable space of parameters relevant for parameter recovery *(1 point)*

*   Use the functions above to generate behavior from a models, for a given set of (randomly sampled) parameters, and then fit the model to its generated data. Make sure to evaluate the parameter fit in a quantiative manner. *(3 points)*

*   Plot the parameter recovery results for both models. *(1 point)*

Make sure to comment your code and provide an explanation for each code block in a preceding text block.





In [ ]:
# YOUR PARAMETER RECOVERY CODE GOES HERE

## *Optional*: Model Recovery *(2 bonus points)*

In this bonus exercise, you may examine model reovery. The bonus points count towards your total group project points. That is, you may accumlate up to 22 points in the practical part of the group project.

Make sure to comment your code and provide an explanation for each code block in a preceding text block.





In [ ]:
# YOUR MODEL RECOVERY CODE GOES HERE

## Model Comparison *(5 points)*

For this exercise you should:

*   Load and (potentially) preprocess the experimental data. (1 point)

*   Fit the two models to the data.  *(1 point)*

*   Evaluate which model performs better, taking into account fit and model complexity. *(2 points)*

*   Plot the behavior of the winning model against the data. *(1 point)**

Make sure to comment your code and provide an explanation for each code block in a preceding text block.





In [ ]:
# YOUR MODEL COMPARISON CODE GOES HERE